In [ ]:
###
# @Author             : Monserrat López
# @Date               : 2025-03-01
# @Last Modified Date : 2025-04-25
# @Description        : Data cleaning, classification, and imputation pipeline for EU data centers.
#                       Includes missing value analysis, rule-based + NLP classification into hyperscale, colocation, and enterprise types,
#                       whitespace imputation strategies, and export of the cleaned dataset for energy modeling.
# @Note               : This script is intended for academic research purposes only. 
#                       Some original raw data collected during the research is not included in this repository for confidentiality reasons.
###

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

### Missing value analysis

In [40]:
# Set the visualization style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

# Load the dataset
df = pd.read_csv('../output/08_DC_EU27.csv')

# Display basic information
print(f"Total data centers: {len(df)}")
print(f"Countries represented: {df['country_normalized'].nunique()}")

# Select the relevant columns for missing value analysis
relevant_vars = [
    'pue_estimate',
    'power_built_out_mw',
    'live_power_mw',
    'whitespace_sqm',
    'building_size_sqm',
    'tier_level',
    'latitude',
    'longitude'
]

# Summary of missing values
missing_summary = df[relevant_vars].isnull().sum().to_frame(name='missing_count')
missing_summary['total'] = len(df)
missing_summary['missing_percent'] = (missing_summary['missing_count'] / missing_summary['total']) * 100
print("Missing values summary:")
print(missing_summary)


Total data centers: 1600
Countries represented: 27
Missing values summary:
                    missing_count  total  missing_percent
pue_estimate                 1513   1600          94.5625
power_built_out_mw           1065   1600          66.5625
live_power_mw                1587   1600          99.1875
whitespace_sqm                976   1600          61.0000
building_size_sqm            1416   1600          88.5000
tier_level                   1166   1600          72.8750
latitude                        0   1600           0.0000
longitude                       0   1600           0.0000


In [41]:
# Distribution of data centers by country
plt.figure(figsize=(14, 8))
country_counts = df['country_normalized'].value_counts()
sns.barplot(x=country_counts.index, y=country_counts.values)
plt.title('Number of Data Centers by Country')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('../figures/datacenter_count_by_country.png', dpi=300)
plt.close()

In [42]:
country_counts

country_normalized
Germany           341
France            221
Netherlands       148
Italy             129
Spain             111
Sweden             76
Ireland            74
Poland             74
Romania            53
Denmark            43
Austria            41
Belgium            37
Czech Republic     36
Portugal           31
Finland            31
Bulgaria           23
Slovenia           17
Croatia            16
Lithuania          16
Latvia             15
Greece             15
Cyprus             11
Hungary            10
Luxembourg          9
Slovakia            9
Estonia             8
Malta               5
Name: count, dtype: int64

### Data Preprocessing

In [43]:
# Prioritize 'live_power_mw' over 'power_built_out_mw' for estimating IT power capacity
df['power_capacity_mw'] = df['live_power_mw'].fillna(df['power_built_out_mw'])

# Prioritize 'whitespace_sqm' over 'building_size_sqm' for usable building area
df['building_area_sqm'] = df['whitespace_sqm'].fillna(df['building_size_sqm'])

# Report remaining missing values
missing_power = df['power_capacity_mw'].isnull().sum()
missing_area = df['building_area_sqm'].isnull().sum()
total = len(df)

print(f"Data centers with missing power capacity: {missing_power} ({missing_power / total * 100:.2f}%)")
print(f"Data centers with missing building area: {missing_area} ({missing_area / total * 100:.2f}%)")

Data centers with missing power capacity: 1065 (66.56%)
Data centers with missing building area: 929 (58.06%)


### NLP Model for categorizing data centres

In [44]:
# Download necessary NLTK resources (uncomment first time)
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')

# Initialize NLTK components
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Prepare a seed dataset with known classifications
# Define keywords and operators associated with each type
dc_types = {
    'hyperscale': {
        'keywords': [
            'hyperscale', 'hyperscaler', 'global cloud', 'cloud-scale', 
            'hyperscale facility', 'hyperscale infrastructure',
            'hyperscale data center', 'hyperscale data centre'
        ],
        'operators': [
            'amazon', 'aws', 'google', 'microsoft', 'azure', 'meta', 'facebook',
            'apple', 'alibaba', 'tencent', 'oracle', 'ibm', 'baidu', 'huawei', 
            'ovh', 'switch', 'cloudflare', 'salesforce'
        ]
    },
    'colocation': {
        'keywords': [
            'colocation', 'co-location', 'colo', 'carrier-neutral', 
            'multi-tenant', 'wholesale', 'neutral facility'
        ],
        'operators': [
            'equinix', 'digital realty', 'ntt', 'cyrusone', 'qts', 'coresite',
            'global switch', 'interxion', 'telehouse', 'colt', 'edgeconnex', 
            'vantage', 'iron mountain', 'atman', 'data4', 'dataplex', 'nexcenter'
        ]
    },
    'enterprise': {
        'keywords': [
            'enterprise', 'private', 'in-house', 'on-premise', 'corporate', 
            'dedicated data center', 'dedicated facility'
        ],
        'operators': []  # Still leave blank
    }
}



In [45]:
# Clean description 
def clean_description(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""

    # Remove common headers and noisy sections
    patterns_to_remove = [
        r'Data Centers[A-Za-z]+\n',        # "Data CentersSpainBarcelona"
        r'Visit Website.*?\n',             # "Visit Website Overview Specs ..."
        r'Overview.*?Request Quote\n',     # Sections
        r'Suites.*?Public Cloud Servers\n', # Lists of services
        r'Colt.*?Data Centre Services.*?\n', # Marketing phrases
        r'AtlasEdge.*?Data Sheet.*?\n',
        r'ISO\s*\d{4,5}',                  # Certifications
        r'Tier\s*\d+',                     # Tier info
        r'(?i)EVENTS.*',                   # Timeline metadata
        r'\bCompany name:.*',              # Company name info
        r'(?i)(colt|atlasedge) [a-z0-9\s,.-]+ acquired',  # Acquisition notes
    ]

    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text, flags=re.DOTALL)

    # Remove emails and URLs
    text = re.sub(r'\S+@\S+', '', text)                     # Emails
    text = re.sub(r'http\S+|www\S+', '', text)              # URLs
    text = re.sub(r'\s+', ' ', text).strip()                # Extra whitespace
    return text

df['cleaned_description'] = df['description'].apply(clean_description)

In [46]:
### Define and apply rule-based classification
def classify_dc_by_rules(row):
    description = str(row.get('cleaned_description', '')).lower()
    operator = str(row.get('operator_name', '')).lower()
    datacenter_name = str(row.get('datacentername', '')).lower()
    combined_text = f"{description} {operator} {datacenter_name}"

    # 1. Hyperscale logic: Require BOTH strong keyword OR strong operator
    hyperscale_match = any(kw in combined_text for kw in dc_types['hyperscale']['keywords'])
    hyperscale_operator = any(op in operator for op in dc_types['hyperscale']['operators'])

    if hyperscale_operator or hyperscale_match:
        return 'hyperscale'

    # 2. Colocation logic: Same (keyword or operator)
    colocation_match = any(kw in combined_text for kw in dc_types['colocation']['keywords'])
    colocation_operator = any(op in operator for op in dc_types['colocation']['operators'])

    if colocation_operator or colocation_match:
        return 'colocation'

    # 3. Enterprise logic: Keywords only (no operator matching needed)
    enterprise_match = any(kw in combined_text for kw in dc_types['enterprise']['keywords'])

    if enterprise_match:
        return 'enterprise'

    # 4. Fallback: Assume enterprise if nothing else matches
    return 'enterprise'

df['dc_type_rule'] = df.apply(classify_dc_by_rules, axis=1)
print("\nInitial Rule-Based Classification:")
print(df['dc_type_rule'].value_counts())


Initial Rule-Based Classification:
dc_type_rule
enterprise    795
colocation    670
hyperscale    135
Name: count, dtype: int64


In [47]:
## Preprocess cleaned text for NLP model
def preprocess_text(text):
    if pd.isna(text) or text == "":
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return " ".join(tokens)

# Combine cleaned text fields
df['combined_text'] = (
    df['cleaned_description'].fillna('') + ' ' +
    df['datacentername'].fillna('') + ' ' +
    df['operator_name'].fillna('')
)

df['processed_text'] = df['combined_text'].apply(preprocess_text)

# Engineered keyword flags
df['has_datacenter_word'] = df['combined_text'].str.contains('data center|datacenter|server|rack', case=False, regex=True).astype(int)
df['has_cloud_word'] = df['combined_text'].str.contains('cloud|aws|azure|gcp', case=False, regex=True).astype(int)
df['has_colo_word'] = df['combined_text'].str.contains('colo|wholesale|multi-tenant', case=False, regex=True).astype(int)
df['has_enterprise_word'] = df['combined_text'].str.contains('enterprise|private|corporate', case=False, regex=True).astype(int)

In [48]:
# Train NLP classification model
# Filter to rows with non-empty processed text
df_with_text = df[df['processed_text'] != ''].copy()

# Use TF-IDF to convert text to numerical features
tfidf = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.7)
text_features = tfidf.fit_transform(df_with_text['processed_text']).toarray()

# Combine with numerical features
numerical_features = df_with_text[
    ['has_datacenter_word', 'has_cloud_word', 'has_colo_word', 'has_enterprise_word']].values
X = np.hstack((text_features, numerical_features))
y = df_with_text['dc_type_rule']  # Using rule-based classification as target

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
print("\nNLP Model Performance:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix for Data Center Type Classification')
plt.tight_layout()
plt.savefig('../figures/dc_type_confusion_matrix.png', dpi=300)
plt.close()


NLP Model Performance:
              precision    recall  f1-score   support

  colocation       0.98      0.97      0.97       129
  enterprise       0.96      1.00      0.98       163
  hyperscale       0.96      0.79      0.86        28

    accuracy                           0.97       320
   macro avg       0.97      0.92      0.94       320
weighted avg       0.97      0.97      0.97       320



In [49]:
# Apply model to all data centers with text
all_with_text = df[df['processed_text'] != ''].copy()
all_text_features = tfidf.transform(all_with_text['processed_text']).toarray()
all_numerical_features = all_with_text[
    ['has_datacenter_word', 'has_cloud_word', 'has_colo_word', 'has_enterprise_word']].values
all_X = np.hstack((all_text_features, all_numerical_features))

# Predict
all_with_text['dc_type_nlp'] = rf_model.predict(all_X)

# Combine rule-based and NLP predictions
df['dc_type_nlp'] = all_with_text['dc_type_nlp']

# For records without text, use rule-based classification
df['dc_type_final'] = df['dc_type_nlp'].fillna(df['dc_type_rule'])

# Count the number of data centers in each category
print("\nFinal Classification:")
print(df['dc_type_final'].value_counts())

# Visualize the classification results
plt.figure(figsize=(10, 6))
df['dc_type_final'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Data Center Classification Results')
plt.xlabel('Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../figures/dc_type_distribution.png', dpi=300)
plt.close()


Final Classification:
dc_type_final
enterprise    801
colocation    669
hyperscale    130
Name: count, dtype: int64


#### Impute whitespace

In [50]:
# Create a working column
df['ws_final'] = df['whitespace_sqm']

# --- Imputation 1: Impute from building size (50%)
mask_building = df['ws_final'].isna() & df['building_size_sqm'].notna()
df.loc[mask_building, 'ws_final'] = df.loc[mask_building, 'building_size_sqm'] * 0.5
print(f"Imputed from building_size_sqm: {mask_building.sum()} rows")

Imputed from building_size_sqm: 47 rows


In [51]:
# --- Compute medians per data centre type
type_medians = (
    df[df['whitespace_sqm'].notna()]
    .groupby('dc_type_final')['whitespace_sqm']
    .median()
    .to_dict()
)
print("Medians per data centre type:", type_medians)

# Impute using type-specific medians
mask_median = df['ws_final'].isna() & df['dc_type_final'].notna()
for t, median_val in type_medians.items():
    t_mask = mask_median & (df['dc_type_final'] == t)
    df.loc[t_mask, 'ws_final'] = median_val
    print(f"Imputed from type-specific median ({t}): {t_mask.sum()} rows")

Medians per data centre type: {'colocation': 1500.0, 'enterprise': 700.0, 'hyperscale': 7250.0}
Imputed from type-specific median (colocation): 292 rows
Imputed from type-specific median (enterprise): 531 rows
Imputed from type-specific median (hyperscale): 106 rows


In [52]:
# Assign the imputed whitespace values to the main column
df['whitespace_sqm'] = df['ws_final']

In [53]:
# Track the source of each imputation BEFORE assigning whitespace_sqm
df['whitespace_source'] = 'original'
df.loc[mask_building, 'whitespace_source'] = 'from_building_size'
df.loc[mask_median, 'whitespace_source'] = 'from_type_median'

In [54]:
# Summary statistics
summary = df.groupby('whitespace_source')['whitespace_sqm'].describe().round(1)
print("\nImputation summary by source:")
print(summary)


Imputation summary by source:
                    count     mean      std    min     25%     50%      75%  \
whitespace_source                                                             
from_building_size   47.0  12376.6  22051.0   45.0  1270.0  5000.0  10980.5   
from_type_median    929.0   1698.8   2025.6  700.0   700.0   700.0   1500.0   
original            624.0   3593.3   8913.8   30.0   400.0  1069.5   3171.5   

                         max  
whitespace_source             
from_building_size  108371.5  
from_type_median      7250.0  
original            110000.0  


In [55]:
# Export cleaned, validated and categorized EU data centers for modeling
df.to_csv("../output/09_classified_DC_EU27.csv", index=False, encoding="utf-8")
print(f"Final cleaned dataset saved with {len(df)} records.")

Final cleaned dataset saved with 1600 records.


In [56]:
missing_whitespace = df['whitespace_sqm'].isna().sum()
total = len(df)
print(f"Missing whitespace entries: {missing_whitespace} ({missing_whitespace / total * 100:.2f}%)")

Missing whitespace entries: 0 (0.00%)


In [57]:
country_counts = df['country_normalized'].value_counts()
country_counts

country_normalized
Germany           341
France            221
Netherlands       148
Italy             129
Spain             111
Sweden             76
Ireland            74
Poland             74
Romania            53
Denmark            43
Austria            41
Belgium            37
Czech Republic     36
Portugal           31
Finland            31
Bulgaria           23
Slovenia           17
Croatia            16
Lithuania          16
Latvia             15
Greece             15
Cyprus             11
Hungary            10
Luxembourg          9
Slovakia            9
Estonia             8
Malta               5
Name: count, dtype: int64